# Language Detection Model

I will try to build Language Detection model using the euclidian distances between n-gram frequency vectors. 

We will need to choose the n value. Here are a few hypotheses:

* 1-grams are just individual charachters and will not retain any sequential information about the data, which seems  important for language detection.

* 2-grams are pairs of chachters. 2 could possibily be an adequate n value, however, 2-grams only capture immediate sequencial information.

* 3-grams or 4-grams intuitively seem to be enough to capture most of the useful sequential information.

* 5-grams and larger may contain many whole words, in effect becoming more like a "dictionary" which will be ineffective when there are words in the test set that are not part of the training set. We can think of this as being analogous to overfitting.

Let's test these.

In [1]:
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

### Let's import the training data.

In [2]:
# Size of traning data to be read in bytes
TRAINING_DATA_SIZE = 2000000

LANGUAGES = [
    'sv', 'da', 'de', 'nl', 'en', 'fr', 'es', 'pt', 'it', 'ro', 'et',
    'fi','lt', 'lv', 'pl', 'sk', 'cs', 'sl', 'hu', 'bg',  'el'
]

# Files names
files = [
    "train/europarl-v7.{lang}-en.{lang}".format(lang=x)
    for x in LANGUAGES
]

# Open files
corpus_raw = [
    open(x).read(TRAINING_DATA_SIZE)
    for x in files
]

Let's remove punctuations. While these probably not entirely useless for language detection, these will increase the number of n-grams which would reduce our confidence in the n-gram frequencies.

In [3]:
corpus = [
    re.sub(r'[?”_"%()!--+,:;./\]\[\xad\n0-9\=<>]', '', x)
    for x in corpus_raw
]

Now we can extract the n-grams from the training data.

In [4]:
#N_VALUES = [1,2,3,4,5,6,7]
N_VALUES = [1,2,3,4,5,6]

count_vectorizers = [
    CountVectorizer(ngram_range=(n, n), analyzer='char_wb')
    for n in N_VALUES
]

counts_array = [
    count_vectorizer.fit_transform(corpus)
    for count_vectorizer in count_vectorizers
]

print("n-values of {} produce {} n-grams repectively".format(
    N_VALUES, list(map(lambda x: x.shape[1], counts_array)
)))

n-values of [1, 2, 3, 4, 5, 6] produce [186, 5957, 61537, 300332, 819272, 1417028] n-grams repectively
Some 3-grams look like this:    (0, 48831)	1
  (0, 110313)	1
  (0, 112085)	1
  (0, 191963)	1
  (0, 119752)	1
  (0, 125742)	1
  (0, 121997)	1
  (0, 151276)	1
  (0, 44845)	1
  (0, 66126)	1
  (0, 197099)	1
  (0, 44817)	1
  (0, 82393)	1
  (0, 102107)	1
  (0, 16801)	1
  (0, 180640)	1
  (0, 124218)	1
  (0, 61091)	1
  (0, 61491)	1
  (0, 54769)	1
  (0, 39844)	1
  (0, 64217)	1
  (0, 178777)	1
  (0, 32269)	1
  (0, 16206)	1
  :	:
  (4, 53765)	261
  (4, 110539)	5782
  (4, 21094)	1655
  (4, 99210)	15
  (4, 23510)	1
  (4, 139281)	1038
  (4, 22735)	29
  (4, 139253)	3
  (4, 151330)	14
  (4, 184085)	2390
  (4, 57982)	2448
  (4, 2200)	2480
  (4, 99337)	25
  (4, 133004)	32
  (4, 81271)	599
  (4, 159998)	3489
  (4, 163016)	2539
  (4, 56479)	318
  (4, 159047)	136
  (4, 7204)	34
  (4, 117849)	6
  (4, 21697)	62
  (4, 63798)	16
  (4, 17896)	14
  (4, 55616)	2


### Calculating Term Weights
$$Term\ Weight = log(Term\ frequency\ in\ a\ particular\ languge\ + 1)
$$

In [ ]:
def compute_weights(counts):
    # Every language should have the same mean count frequency
    counts = counts/counts.mean(axis=1)
        
    # Sublinear transform to ensure no terms have overwhelmingly large weight
    counts = np.log(counts+1)

    return counts

language_weights = [compute_weights(c) for c in counts_array]
analyzers = [count_vectorizer.build_analyzer() for count_vectorizer in count_vectorizers]

## Socring test samples
 We can think of test samples as points in space in the same number of dimentions as our training data. We will treat test samples exactly the same way as our training set. Our prediction will simply be the language with the lowest Euclidean distance from the test sample.


In [ ]:
def predict(text, n):
    term_weight = compute_weights(count_vectorizers[n-1].transform([text]))    
    distances = [np.linalg.norm(term_weight-language_weight) for language_weight in language_weights[n-1]]
    return LANGUAGES[np.argmin(distances)]

def run_tests(n):
    print("\nRuning test for n = {}".format(n))
    right = 0
    wrong = 0
    tests = open('europarl.test')
    for x in range(TRAINING_DATA_SIZE):
        line = tests.readline()
        if line == '':
            print("Error rate: {}%".format(100*wrong/(right+wrong)))
            return
        [lang, text] = line.split('\t')
        if predict(text, n) == lang:
            right = right + 1
        else:
            wrong = wrong + 1

for n in N_VALUES:
    run_tests(n)


Runing test for n = 1
Final er: 5.371428571428571%
total right: 19872 
total wrong: 1128

Runing test for n = 2
Final er: 7.719047619047619%
total right: 19379 
total wrong: 1621

Runing test for n = 3
Final er: 43.31428571428572%
total right: 11904 
total wrong: 9096

Runing test for n = 4
Final er: 66.94285714285714%
total right: 6942 
total wrong: 14058

Runing test for n = 5


N = 1 gives us the best error rate. This is odd since N = 1 takes into account only charchter frequencies. Let look at what is going on. 

In [ ]:
count_vectorizers[0].transform(["This is an english sentence"]).todense()

It is clear that test samples are very small and will comatin mostly zeros. This is a very low signal to noise ratio. The distances between zero and the actual language values for these dimentions do not provide much information useful for classification. Let's see what would happen if we can consider another space consisting only of dimentions that exist in the test sample.

In [ ]:
def predict(text, n):
    term_weight = compute_weights(count_vectorizers[n-1].transform([text]))
    
    # Inculde only dimentions where term wieght is not zero
    relevent_dimentions = [
        language_weight[term_weights != 0] for language_weight in language_weights[n-1]
    ]
    term_weight = term_weights[term_weight != 0]
    
    distances = [np.linalg.norm(term_weights-c) for c in relevent_dimentions]
    return LANGUAGES[np.argmin(distances)]


